In [1]:
!pip install chatterbot
!pip install chatterbot_corpus


     |████████████████████████████████| 71kB 4.7MB/s 
     |████████████████████████████████| 122kB 8.6MB/s 


In [2]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
import re
from chatterbot.trainers import ChatterBotCorpusTrainer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
from tensorflow.keras.models import Model 
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
import random

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
df = pd.read_excel('DFI -Ticketing.xlsx', sheet_name=None)

df = pd.DataFrame(df['Export - Tickets'])


In [4]:
df = pd.read_csv('sample.csv', encoding='cp1252', low_memory=False)


In [5]:
x= df[['Incident-Subject','Team']]
x.head()

Incident-Subject               Team
0   Bonsoir     Notre outil de supervision a remo...           PILOTAGE
1   Bonjour,     Notre plateforme de supervision ...           PILOTAGE
2   Bonjour,     Message reçu dans QSYSOPR      T...           PILOTAGE
3   Bonjour,     Notre outil de supervision nous ...  SUPPORT BT NETAPP
4   Bonjour,     Host: FRVE-SRV-H34VM  @NAT : 10....           PILOTAGE

In [6]:
df.loc[df["Team"]=="SUPPORT BT NETAPP"]

ID           Structure  ...     Category Sub-Category
3     55023                D-FI  ...          NaN          NaN
1580  57291                D-FI  ...          NaN          NaN
2289  58469                D-FI  ...          NaN          NaN
2362  58574                D-FI  ...          NaN          NaN
2363  58575                D-FI  ...          NaN          NaN
2653  59021  POCLAIN HYDRAULICS  ...          NaN          NaN
2685  59090                D-FI  ...          NaN          NaN
3164  59827  POCLAIN HYDRAULICS  ...          NaN          NaN
3358  60121  NUTRITION ET SANTE  ...        Autre          NaN
3854  60884  NUTRITION ET SANTE  ...        Autre          NaN
4125  61268  NUTRITION ET SANTE  ...        Autre          NaN
4498  61792  POCLAIN HYDRAULICS  ...          NaN          NaN
5892  63760                D-FI  ...          NaN          NaN
5958  63856                D-FI  ...          NaN          NaN
7368  65786  NUTRITION ET SANTE  ...  Supervision          NaN
7985  66625                D-FI  ...          NaN          NaN
8325  67074                D-FI  ...          NaN          NaN
9959  69222                D-FI  ...          NaN          NaN

[18 rows x 29 columns]

In [7]:
tfidf = TfidfVectorizer()

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df["Incident-Subject"].loc[df["Team"]=="SUPPORT BT NETAPP"])
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

In [8]:
class submit_proposal():

  def __init__(self,df,model,tokenizer,labels):

    self.df = df
    self.model = model
    self.tokenizer = tokenizer
    self.labels = labels

  def predict_team(self,question):

    new_model = tf.keras.models.load_model(self.model)

    with open(self.tokenizer, 'rb') as handle:
      tokenizer = pickle.load(handle)

    with open(self.labels, 'rb') as handle:
        lables_columns = pickle.load(handle)

    question=np.array(question)
    question = question.reshape(-1)
    question = tokenizer.texts_to_sequences(question)
    question = pad_sequences(question)
    prediction =    lables_columns[np.argmax(new_model.predict(question))]

    return prediction


  def answer_ticket(self,prediction,question):

    df_team = self.df.loc[self.df["Team"]==prediction]
    tickets_questions = df_team['Incident-Subject'].copy()
    tickets_questions = tickets_questions.to_frame()

    x=str(question)
    
    x = pd.DataFrame([[x]], columns=['Incident-Subject'])
    tickets_questions=tickets_questions.append(x, ignore_index=True)

    #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
    tfidf = TfidfVectorizer()
    
    #Construct the required TF-IDF matrix by fitting and transforming the data
    tfidf_matrix = tfidf.fit_transform(tickets_questions['Incident-Subject'])
    cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)
    sim_scores = list(enumerate(cosine_similarity[-1]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    max_scores = [i[0] for i in sim_scores if i[0]<len(sim_scores)-1]
    max_scores = max_scores[0:5]
    proposals = self.df['Ticket-Description'][max_scores]
    return proposals
  def train_chatbot(self):

    chatbot = ChatBot('ticket_bot'  ,  logic_adapters=[
    'chatterbot.logic.BestMatch',
    'chatterbot.logic.TimeLogicAdapter'],)

    # First, lets train our bot with some data
    trainer = ChatterBotCorpusTrainer(chatbot)
    trainer.train('chatterbot.corpus.french')
    #'''
    trainer = ListTrainer(chatbot)

    trainer.train([
    'bonjour',
    'bonjour',
    'bonsoir',
    'bonsoir',
    'J''ai un problème',
    #'avez-vous des problèmes?',
    #'ou',
    's''il vous plaît, donnez-moi votre billet',
    'Merci',
    ])
    #'''
    return chatbot
  def chatbot(self):
    print("Bonjour, entre au revoir pour exister")

    search=False
    chatbot = self.train_chatbot()
    while True:
        Inquiry=input('moi'+':')
        #print(Inquiry)
        exist = re.findall(r'revoir|Revoir',Inquiry)
        yes  = re.findall(r'problème|billet|problèmes|Problèmes|Problème|Billet',Inquiry)
        if search:
          answer = "la solution de votre ticket est"
          search = False
          
          team = self.predict_team(Inquiry[3:])
          print(team)
          proposals = self.answer_ticket(team,Inquiry[3:])
          proposals = proposals.values
          print('Mo-Bot:',answer+proposals[0])
          continue

        if len(yes) >0:
          search = True
          answer=chatbot.get_response('J''ai un problème')
          answer = 's''il vous plaît, donnez-moi votre billet'
          print('Mo-Bot:',answer)
          
        #print(exist)
        if len(exist) >0 :
          break
        if not search:
          answer=chatbot.get_response(Inquiry[3:])
          print('Mo-Bot:',answer)



In [10]:
model='/content/my_model.h5'
tokenizer = '/content/tokenizer.pickle'
labels = '/content/lables.pickle'
question = ' Bonjour, Nous avon un problème de transfert de données QUICKEDD entre la PROD (TECHM3) et le backup(BKTECHM3).  Aucun transfert ne s''effectue et nous avons 18h00 de retard. '

pro = submit_proposal(df,model,tokenizer,labels)
team = pro.predict_team(question)
proposals = pro.answer_ticket(team,question)

pro.chatbot()


Bonjour, entre au revoir pour exister
Training botprofile.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training food.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training trivia.yml: [####################] 100%
List Trainer: [####################] 100%
moi:bonjour
Mo-Bot: Salut
moi:J'ai un problème
Mo-Bot: sil vous plaît, donnez-moi votre billet
moi:' Bonjour, Nous avon un problème de transfert de données QUICKEDD entre la PROD (TECHM3) et le backup(BKTECHM3).  Aucun transfert ne s''effectue et nous avons 18h00 de retard. '
PILOTAGE
Mo-Bot: la solution de votre ticket est Dear Jorge,     Happy new year for you and all your families.     ESBAWSUS1 yearly backup has just finished.     You can add tapes for the daily backup.     Many thanks.        Nabil AMER  Administrateur Sauvegarde     nam@d-fi.fr <mailto:nam@d-fi.fr>           2 rue de Paris 92190 MEUDON  RCS NANTERRE 333903128  Site web : www.d-fi.fr